# Copy the raw data to a local temp file

In [0]:
# Set the ADLS Gen2 folder path containing CSV files
csv_silver_path = "abfss://silver@qydatalake.dfs.core.windows.net/dynamicYeild/productfeed_csv/"

# List all files in the folder and filter for those ending with '.csv'
files = [file.name for file in dbutils.fs.ls(csv_silver_path) if file.name.endswith('.csv')]

# If no CSV files are found, raise an error to halt execution
if not files:
    raise FileNotFoundError(f"No CSV files found in {csv_silver_path}")

# Construct the full source path to the first CSV file (the only one CSV file in the folder)
source_file = csv_silver_path + files[0]

# Define the local temporary file path on the driver node
tmp_file = "file:///tmp/productfeed.csv"

# Copy the CSV from ADLS to the local temp directory
dbutils.fs.cp(source_file, tmp_file)

True

# Upload the local temp file to S3

In [0]:
import boto3

# Initialize the S3 client using AWS credentials stored in Azure Key Vault
s3 = boto3.client(
    "s3",
    aws_access_key_id = dbutils.secrets.get(scope="azure_key_vault", key="DY-AWS-S3-KEY"),
    aws_secret_access_key = dbutils.secrets.get(scope="azure_key_vault", key="DY-AWS-S3-SECRET"),
    region_name = "us-east-1"
)

# Define the target S3 bucket and object key (folder path + filename)
bucket = "com.dynamicyield.feeds"
key = "8776216/productfeed.csv"

# Define the local file path of the CSV to upload
local_file = "/tmp/productfeed.csv"

# Upload the local CSV file to the specified S3 bucket and key
s3.upload_file(local_file, bucket, key)

# Validate upload status

In [0]:
import botocore

try:
    # Attempt to fetch the S3 object’s metadata to confirm it exists
    resp = s3.head_object(Bucket=bucket, Key="8776216/productfeed.csv")

    # If successful, print a confirmation with the last modified timestamp
    print("Uploaded product feed to DY at:")
    print(resp["LastModified"])
except botocore.exceptions.ClientError as e:
    # If there’s a ClientError (e.g., 404 Not Found), alert and re-raise the exception
    print("Failed to upload product feed to DY!")
    raise

Uploaded product feed to DY at:
2025-07-03 00:36:24+00:00
